In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
# Load Data
df = pd.read_csv('data/BrentOilPrices_Cleaned.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)

In [ ]:
### ARIMA Model Training
model = ARIMA(df['Brent_Oil_Price'], order=(5,1,0))
fitted_model = model.fit()
forecast = fitted_model.forecast(steps=30)

In [ ]:
# Plot ARIMA Forecast
plt.figure(figsize=(12,5))
plt.plot(df.index, df['Brent_Oil_Price'], label="Actual Prices")
plt.plot(pd.date_range(df.index[-1], periods=30, freq='D'), forecast, label="ARIMA Forecast", linestyle='dashed', color='red')
plt.xlabel("Year")
plt.ylabel("Price (USD)")
plt.title("Brent Oil Price ARIMA Forecast")
plt.legend()
plt.show()

In [ ]:
### LSTM Model Training
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df[['Brent_Oil_Price']])

In [ ]:
# Prepare Training Data
X, y = [], []
for i in range(len(df_scaled)-10):
    X.append(df_scaled[i:i+10])
    y.append(df_scaled[i+10])

X, y = np.array(X), np.array(y)

In [ ]:
# Define LSTM Model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(10,1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=50, verbose=1)

print("LSTM Model Training Complete")